<a href="https://colab.research.google.com/github/nhuyen183/DS-DV-Project/blob/master/LungCancerDSS_preprocessing25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,563 kB]
Get:13 http://se

# Create Spark entry points


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

In [4]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest

In [ ]:
# load packages
import sys
print('Python version: {}'. format(sys.version))

import pandas as pd
print('Python version: {}'. format(pd.__version__))

import matplotlib
print('matplotlib version: {}'. format(matplotlib.__version__))

import numpy as np
print('numpy version: {}'. format(np.__version__))

import scipy as sp
print('scipy version: {}'. format(sp.__version__))

import IPython
from IPython import display # pretty printing of dataframe in Jupyter notebook
print('IPython version: {}'. format(IPython.__version__))

import pyspark
print('Apache Spark Pyspark version: {}'. format(pyspark.__version__)) # pyspark version

# misc libraries
import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Python version: 1.3.5
matplotlib version: 3.2.2
numpy version: 1.21.6
scipy version: 1.7.3
IPython version: 7.9.0
Apache Spark Pyspark version: 3.1.1
-------------------------


In [16]:
data_2020 = spark.read.csv('./brfss2020.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2020.dtypes)

---------- data types ----------


,0,1
0,_STATE,double
1,FMONTH,double
2,IDATE,int
3,IMONTH,int
4,IDAY,int
...,...,...
274,_STOLDNA,double
275,_VIRCOLN,double
276,_SBONTIM,double
277,_CRCREC1,double


In [17]:
data_2020.show(6)

+------+------+-------+------+----+-----+--------+----------+-------------+--------+--------+--------+--------+--------+-------+-------+--------+-------+------+--------+--------+--------+--------+--------+-------+-------+--------+--------+-------+--------+-------+------+-------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-----+--------+--------+--------+--------+--------+-------+--------+-------+--------+-------+-------+----+-----+------+--------+--------+--------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+-------+-------+--------+-------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--

In [26]:
data_2020F = data_2020.select('SEXVAR', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3') #'ECIGARET',  'ECIGNOW'
data_2020F.show(6)

+------+--------+--------+-------+--------+--------+--------+-------+--------+--------+--------+--------+-------+
|SEXVAR|_AGE65YR|_BMI5CAT|GENHLTH|SMOKE100|_SMOKER3|LCSFIRST|LCSLAST|LCSNUMCG|LCSCTSCN|CNCRTYP1|STOPSMK2|ASTHMA3|
+------+--------+--------+-------+--------+--------+--------+-------+--------+--------+--------+--------+-------+
|   2.0|     1.0|     1.0|    2.0|     1.0|     1.0|    null|   null|    null|    null|    null|     2.0|    1.0|
|   2.0|     2.0|     3.0|    3.0|    null|     9.0|    null|   null|    null|    null|    null|    null|    1.0|
|   2.0|     2.0|    null|    3.0|     2.0|     4.0|    null|   null|    null|    null|    null|    null|    2.0|
|   2.0|     2.0|    null|    1.0|     2.0|     4.0|    null|   null|    null|    null|    null|    null|    2.0|
|   2.0|     2.0|     2.0|    2.0|     2.0|     4.0|    null|   null|    null|    null|    null|    null|    2.0|
|   1.0|     2.0|     3.0|    4.0|     1.0|     3.0|    null|   null|    null|    null| 

In [20]:
data_2019 = spark.read.csv('./2019.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_2019.dtypes)

---------- data types ----------


,0,1
0,_c0,int
1,_STATE,int
2,FMONTH,int
3,IDATE,int
4,IMONTH,int
...,...,...
338,_FRUITE1,int
339,_VEGETE1,int
340,_FLSHOT7,string
341,_PNEUMO3,string


In [23]:
data_2019F = data_2019.select('SEXVAR', '_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3')
data_2019F.show(6)

+------+--------+--------+-------+--------+--------+--------+-------+--------+--------+--------+--------+-------+
|SEXVAR|_AGE65YR|_BMI5CAT|GENHLTH|SMOKE100|_SMOKER3|LCSFIRST|LCSLAST|LCSNUMCG|LCSCTSCN|CNCRTYP1|STOPSMK2|ASTHMA3|
+------+--------+--------+-------+--------+--------+--------+-------+--------+--------+--------+--------+-------+
|     2|       2|       3|      3|       1|       3|      NA|     NA|      NA|      NA|      NA|      NA|      2|
|     2|       2|       2|      4|       2|       4|      NA|     NA|      NA|      NA|      NA|      NA|      2|
|     2|       2|       4|      3|       2|       4|      NA|     NA|      NA|      NA|      NA|      NA|      2|
|     2|       2|       2|      4|      NA|       9|      NA|     NA|      NA|      NA|      NA|      NA|      2|
|     2|       2|       2|      2|       1|       3|      NA|     NA|      NA|      NA|      NA|      NA|      2|
|     2|       3|      NA|      2|      NA|       9|      NA|     NA|      NA|      NA| 

In [2]:
data_2019F.dtypes

NameError: ignored

In [27]:
data_raw = data_2019F.union(data_2020F)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_raw.dtypes)

---------- data types ----------


,0,1
0,SEXVAR,double
1,_AGE65YR,double
2,_BMI5CAT,string
3,GENHLTH,string
4,SMOKE100,string
...,...,...
8,LCSNUMCG,string
9,LCSCTSCN,string
10,CNCRTYP1,string
11,STOPSMK2,string


In [ ]:
# data summary
print('-'*10, 'data summary', '-'*10)
data_raw.describe().toPandas()

---------- data summary ----------


,summary,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,...,_RFPSA23,_CLNSCPY,_SGMSCPY,_SGMS10Y,_RFBLDS4,_STOLDNA,_VIRCOLN,_SBONTIM,_CRCREC1,_AIDTST4
0,count,401958,401958,401958,401958,401958,401958,401958,401958,401958,...,126604,177922,171486,171486,175853,176357,174274,194217,175698,367921
1,mean,30.010359291269236,5.959192253917076,6662701.046606859,6.510600610013982,15.008040641062001,2020.0301822578479,1117.7478741560064,2.0200045922818801E9,2.0200045922818801E9,...,2.3371615430792074,1.5802655096053326,2.851958760481905,2.8366630512111777,2.57898926944664,2.880906343383024,2.9384589783903508,2.5372959112745024,1.4172785119921685,2.040280386278576
2,stddev,15.773473494903223,3.5242238117738207,3462450.1811220297,3.4669000464972615,8.428138498040536,0.1710887546889567,38.207381620255546,3396.6841138105215,3396.6841138105215,...,2.2033784158573253,0.8728051431053163,0.44380827118133553,0.4879450778036646,0.6730066711378386,0.4513059766839771,0.32518874638973705,0.5119272764015419,0.7627272276885876,1.680522189372081
3,min,1.0,1.0,1012021,1,1,2020,1100.0,2020000001,2.020000001E9,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,max,72.0,12.0,12312020,12,31,2021,1200.0,2020015835,2.020015835E9,...,9.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,9.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Define the problem
What sorts of people were likely to have lung cancer?

# Step 2: Gather the data
The datasets can be found here: https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data

# Step 3: Prepare data for consumption

In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download aemreusta/brfss-2020-survey-data

 97% 47.0M/48.3M [00:02<00:00, 20.1MB/s]
100% 48.3M/48.3M [00:02<00:00, 19.5MB/s]


In [7]:
! kaggle datasets download sakinak/behavioral-risk-factor-surveillance-survey-201619

 99% 231M/234M [00:07<00:00, 36.9MB/s]
100% 234M/234M [00:07<00:00, 31.3MB/s]


In [8]:
!ls

behavioral-risk-factor-surveillance-survey-201619.zip  drive
brfss-2020-survey-data.zip			       sample_data


In [9]:
!unzip brfss-2020-survey-data.zip

Archive:  brfss-2020-survey-data.zip
  inflating: brfss2020.csv           


In [10]:
!unzip behavioral-risk-factor-surveillance-survey-201619.zip

Archive:  behavioral-risk-factor-surveillance-survey-201619.zip
  inflating: 2016.csv                
  inflating: 2017.csv                
  inflating: 2018.csv                
  inflating: 2019.csv                


In [11]:
from subprocess import check_output
print('-'*10, 'Files', '-'*10)
print(check_output(['ls', './']).decode('utf8'))

---------- Files ----------
2016.csv
2017.csv
2018.csv
2019.csv
behavioral-risk-factor-surveillance-survey-201619.zip
brfss2020.csv
brfss-2020-survey-data.zip
drive
sample_data



## About the BRFSS Dataset and Prediction Task

The Behavioral Risk Factor Surveillance System (BRFSS) is a collaborative project between all of the states in the United States and participating US territories and the Centers for Disease Control and Prevention (CDC).

BRFSS’s objective is to collect uniform state-specific data on health risk behaviors, chronic diseases and conditions, access to health care, and use of preventive health services related to the leading causes of death and disability in the United States. BRFSS conducts both landline and mobile phone-based surveys with individuals over the age of 18. General factors assessed by the BRFSS in 2020 included health status and healthy days, exercise, insufficient sleep, chronic health conditions, oral health, tobacco use, cancer screenings, and access to healthcare.

The aim of this project is to build a model with relatively high accuracy and AUC that could serve as an decision aid for those at high risk of developing lung cancer.

The data contains information about 401958 unique survey participant. As a result of my research to select the ones related to coronary artery disease among a total of 279 different features. Each example in the dataset contains the following demographic data for a set of individuals

### Categorical Features
*   `_AGE65YR`: The age of the individual in years two-level categories `18 <= AGE <= 64`: `1` and `65 <= AGE <= 99`:`2`
*   `SEXVAR`: Sex of Respondent `Male: 1` and `Female: 2`
*   `_BMI5CAT`:  Four-categories of Body Mass Index (BMI)`_BMI5 < 1850: Underweight` ; `1850 <= _BMI5 < 2500: Normal`;`2500 <= _BMI5 < 3000: Overweight`;`3000 <= _BMI5 < 9999: Obese`
*   `GENHLTH`: Health status: Would you say that in general your health is: `1: Excellent`; `2: Very good` ; `3: Good` ; `4: Fair` ; `5: Poor`
*   `SMOKE100`: Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes] `1: Yes` ; `2: No`
*   `_SMOKER3`: Four-level smoker status: Everyday smoker: `1`, Someday smoker: `2`, Former smoker: `3`, Non-smoker: `4`

### Lung Cancer (Features) Screening Section
*   `LCSFIRST`: How old were you when you first started to smoke cigarettes regularly. `Value 1-100 in years`
*   `LCSLAST`: How old were you when you last smoked cigarettes regularly? `Value 1-100 in years`
*   `LCSNUMCG`: On average, when you smoke/smoked regularly, about how many cigarettes do/did you usually smoke each 
day? `Value 1-300 in number of cigarettes`
*   `LCSCTSCN`: In the last 12 months, did you have a CT or CAT scan? Example include: `Yes, to check for lung cancer`, `No (did not have a CT scan`, `Had a CT scan, but for other reason`.
*   `CNCRTYP1`:  What type of cancer was it? (If Response = 2 (Two) or 3 (Three or more), ask: “With your most recent 
diagnoses of cancer, what type of cancer was it?”). Examples include: `Lung cancer: 24`, `Others: 1-30`
*   `STOPSMK2`:  During the past 12 months, have you stopped smoking for one day or longer because you were trying to quit smoking? `Yes: 1` or `No: 2`.
*   `ECIGARET`: Have you ever used an e-cigarette or other electronic vaping product, even just one time, in your entire life? `Yes: 1` or `No: 2`.
*   `ECIGNOW`: Do you now use e-cigarettes or other electronic vaping products every day, some days, or not at all? `Every day: 1`; `Some days: 2` or `Not at all: 3`
* `ASTHMA3`: (Ever told) (you had) asthma? `Yes: 1` or `No: 2`.
### Prediction Task
The prediction task is to **early predict whether a person have the high risk of lung cancer.**

### Label
*   `CNCRTYP1`: What type of cancer (lung cancer = 24)

### Notes on Data Collection

All the examples extracted for this dataset meet the following conditions: 
*   `age` is 16 years or older.
*   The adjusted gross income (used to calculate `income_bracket`) is greater than $100 USD annually.
*   `fnlwgt` is greater than 0.
*   `hours_per_week` is greater than 0.






In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
import seaborn as sns

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

from google.colab import widgets
# For facets
from IPython.core.display import display, HTML
import base64
!pip install facets-overview==1.0.0
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from glob import glob
files = glob("./*.csv")
files

['./2016.csv', './brfss2020.csv', './2018.csv', './2019.csv', './2017.csv']

In [ ]:
df = pd.concat([pd.read_csv(file) for file in files])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (131) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (125) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (171) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [29]:
columns = ['_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3'] #'SEXVAR', 

data_2020 = pd.read_csv('./brfss2020.csv', usecols=columns).dropna().astype("float")
print("df shape:", data_2020.shape)
data_2020.head()

df shape: (236, 12)


,GENHLTH,ASTHMA3,SMOKE100,STOPSMK2,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSCN,CNCRTYP1,_AGE65YR,_BMI5CAT,_SMOKER3
48625,3.0,2.0,1.0,1.0,14.0,39.0,4.0,3.0,22.0,1.0,4.0,2.0
48647,3.0,2.0,1.0,2.0,18.0,66.0,10.0,2.0,4.0,2.0,4.0,1.0
48691,3.0,2.0,1.0,2.0,18.0,70.0,10.0,2.0,30.0,2.0,1.0,1.0
48698,1.0,2.0,1.0,1.0,21.0,71.0,10.0,1.0,22.0,2.0,4.0,1.0
48719,5.0,2.0,1.0,2.0,13.0,75.0,777.0,2.0,22.0,2.0,3.0,1.0


In [26]:
columns = ['_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3'] #'SEXVAR'

data_2019 = pd.read_csv('./2019.csv', usecols=columns).dropna().astype("float")
print("df shape:", data_2019.shape)
data_2019.head()

df shape: (0, 12)


,GENHLTH,ASTHMA3,SMOKE100,STOPSMK2,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSCN,CNCRTYP1,_AGE65YR,_BMI5CAT,_SMOKER3


In [24]:
columns = ['_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3'] #'SEXVAR'

data_2018 = pd.read_csv('./2018.csv', usecols=columns).dropna().astype("float")
print("df shape:", data_2018.shape)
data_2018.head()

df shape: (240, 12)


,GENHLTH,ASTHMA3,SMOKE100,STOPSMK2,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSCN,CNCRTYP1,_AGE65YR,_BMI5CAT,_SMOKER3
54354,2.0,2.0,1.0,1.0,777.0,777.0,10.0,2.0,22.0,1.0,2.0,2.0
54389,2.0,2.0,1.0,2.0,15.0,67.0,30.0,2.0,22.0,2.0,2.0,1.0
54432,1.0,2.0,1.0,2.0,15.0,50.0,20.0,3.0,21.0,1.0,4.0,1.0
54512,3.0,2.0,1.0,2.0,15.0,63.0,20.0,2.0,22.0,1.0,2.0,1.0
54597,2.0,2.0,1.0,1.0,15.0,64.0,10.0,1.0,22.0,1.0,1.0,1.0


In [25]:
columns = ['_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN', 'CNCRTYP1',
                  'STOPSMK2', 'ASTHMA3'] #'SEXVAR'

data_2017 = pd.read_csv('./2017.csv', usecols=columns).dropna().astype("float")
print("df shape:", data_2017.shape)
data_2017.head()

df shape: (95, 12)


,GENHLTH,ASTHMA3,SMOKE100,STOPSMK2,CNCRTYP1,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSCN,_AGE65YR,_BMI5CAT,_SMOKER3
90783,2.0,1.0,1.0,1.0,22.0,57.0,61.0,10.0,2.0,1.0,3.0,1.0
90794,4.0,2.0,1.0,1.0,24.0,18.0,60.0,5.0,1.0,2.0,1.0,2.0
90854,5.0,2.0,1.0,2.0,21.0,15.0,64.0,40.0,3.0,1.0,2.0,1.0
90862,5.0,2.0,1.0,1.0,30.0,19.0,73.0,15.0,1.0,2.0,2.0,1.0
90873,1.0,2.0,1.0,1.0,19.0,20.0,58.0,6.0,3.0,1.0,3.0,2.0


In [30]:
df_selected = pd.concat([data_2017, data_2018, data_2020])
print("df shape:", df_selected.shape)
df_selected.head()

df shape: (571, 12)


,GENHLTH,ASTHMA3,SMOKE100,STOPSMK2,CNCRTYP1,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSCN,_AGE65YR,_BMI5CAT,_SMOKER3
90783,2.0,1.0,1.0,1.0,22.0,57.0,61.0,10.0,2.0,1.0,3.0,1.0
90794,4.0,2.0,1.0,1.0,24.0,18.0,60.0,5.0,1.0,2.0,1.0,2.0
90854,5.0,2.0,1.0,2.0,21.0,15.0,64.0,40.0,3.0,1.0,2.0,1.0
90862,5.0,2.0,1.0,1.0,30.0,19.0,73.0,15.0,1.0,2.0,2.0,1.0
90873,1.0,2.0,1.0,1.0,19.0,20.0,58.0,6.0,3.0,1.0,3.0,2.0


In [31]:
features = ['_AGE65YR', '_BMI5CAT', 'GENHLTH', 'SMOKE100', '_SMOKER3',
                  'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSCN',
                  'STOPSMK2', 'ASTHMA3'] #'SEXVAR',  'ECIGARET', 'ECIGNOW'
X = df_selected[features]
y = df_selected['CNCRTYP1']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)

In [32]:
X_train.shape

(364, 11)

In [33]:
y_train.shape

(364,)

## Analyzing the BRFSS Dataset with Facets
It is important to understand your dataset *before* diving straight into the prediction task. 

Some important questions to investigate when auditing a dataset for fairness:

* **Are there missing feature values for a large number of observations?**
* **Are there features that are missing that might affect other features?**
* **Are there any unexpected feature values?**
* **What signs of data skew do you see?**

In [34]:
#@title Visualize the Data in Facets
fsg = FeatureStatisticsGenerator()
dataframes = [
    {'table': df_selected, 'name': 'trainData'}]
censusProto = fsg.ProtoFromDataFrames(dataframes)
protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")


HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))

**SEXVAR**

SEXVAR column contains participants gender information. 1 means male and 2 means female. For better reading, the column renamed as 'Gender'.

In [ ]:
df_selected['SEXVAR'].value_counts()
df_selected = df_selected.rename(columns={"SEXVAR":"Gender"})
sns.countplot(x='Gender', data=df_selected)

**_AGE65YR**

The _AGE65YR column contains information about the age of the participants. Participants aged 65 and older are marked as 2, while younger ones are marked as 1. 3 means missing values and they removed.

For better reading, the column renamed as 'Over 65 Years Old'.

In [ ]:
df_selected['_AGE65YR'].value_counts()
df_selected.drop(df_selected[df_selected['_AGE65YR'] == 3].index, inplace = True)
df_selected = df_selected.rename(columns={"_AGE65YR":"Over 65 Years Old"})
sns.countplot(x='Over 65 Years Old', hue='Gender', data=df_selected)

**GENHLTH**

The GENHLTH column contains participants general health information.

1 means excellent;
2 means very good;
3 means good;
4 means fair;
5 means poor;
7 and 9 means Don’t know/Not Sure/Refused

For better reading, the column renamed as 'General Health Status'. Uncertain values removed.

In [ ]:
df_selected['GENHLTH'].value_counts()
df_selected = df_selected[(df_selected['GENHLTH'] != 7) & (df_selected['GENHLTH'] != 9)]
df_selected = df_selected.rename(columns={"GENHLTH":"General Health Status"})
sns.countplot(x='General Health Status', hue='Gender', data=df_selected)

In [37]:
#@title Set the Number of Data Points to Visualize in Facets Dive

SAMPLE_SIZE = 61 #@param
  
train_dive = X_train.sample(SAMPLE_SIZE).to_json(orient='records')

HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=train_dive)
display(HTML(html))